## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

In [2]:
from Functions import (gdp_future, pop_func, emis_func, determine_coefficient,
                       popshare_func, emisshare_func, emis_total_func, rho, cumpopshare_func,
                       cumfuturepop_func, emis_f_func)

## Parameter

In [3]:
hist_startyear = 1950

## Paths

In [4]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF/WP5/")
path_data = path_main / "Data" / "EffortSharing" / str(hist_startyear)
path_figs = path_main / "Figures"

## Read data files

In [5]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [6]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [7]:
df_countries = pd.read_csv("X:/user/dekkerm/Data/AllCountries.csv")

## Preprocess

In [8]:
ndc_deviations = []
for c_i, c in enumerate(["C1", "C3", "C5"]):
    li = []
    for a_i, approach in enumerate(["GF", "PC", "PCC"]):
        li.append([float((xr_budgets_scenario.sel(Ccat = c, Variable = approach, Region=r, Time=2030).Value - ((xr_total.NDC_l.sel(Time=2030, ISO=r)+xr_total.NDC_h.sel(Time=2030, ISO=r))/2)) / ((xr_total.NDC_l.sel(Time=2030, ISO=r)+xr_total.NDC_h.sel(Time=2030, ISO=r))/2)) for r in xr_budgets_scenario.Region])
    ndc_deviations.append(li)
            

In [9]:
ar_name = np.array(df_countries['alpha-3'])
ar_reg = np.array(df_countries["sub-region"])
regions = []
for c in all_countries_iso:
    subreg = ar_reg[ar_name == c]
    if len(subreg) == 0:
        subreg = ["nan"]
    regions.append(subreg[0])
regions = np.array(regions).astype(str)

## Plot

In [10]:
len(np.unique(regions))

18

In [11]:
df = {}
df["ISO"] = np.copy(all_countries_iso)
df["CONT"] = regions
for app in ["GF", "PC", "PCC"]:
    gf = np.array(xr_budgets_static.sel(Ccat="C1", Variable=app, Region=all_countries_iso).Value)
    gf[np.isnan(gf)] = 1e-9
    df[app] = gf
df = pd.DataFrame(df)

In [12]:
def build_hierarchical_dataframe(df, levels, value_column, appname, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = appname
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id=appname, parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees

In [13]:
dfs= []
for app_i, app in enumerate(["GF", "PC", "PCC"]):
    appname = ["Grandfathering", "Per Capita", "Per Capita Convergence"][app_i]
    df_hier = build_hierarchical_dataframe(df, ["ISO","CONT"], app, appname, [app, app])
    # color = (df_hier['value'] - np.min(df[app])) / (np.max(df[app])- np.min(df[app]))
    # wh_reg = np.where(df_hier.parent == "total")[0]
    # color[wh_reg] = (color[wh_reg] - np.min(color[wh_reg])) / (np.max(color[wh_reg]) - np.min(color[wh_reg]))
    # color[df_hier.parent == ''] = np.nan
    color = np.array([0.5]*len(df_hier))
    for r_i, r in enumerate(np.unique(regions)):
        wh_reg = np.where((df_hier.id == r) | (df_hier.parent == r))[0]
        color[wh_reg] = r_i/18
    # for r_i, r in enumerate(np.unique(regions)):
    #     wh_reg = np.where((df_hier.parent == r))[0]
    #     for wh in wh_reg:
    #         if df_hier.id[wh] in ["CHN", "USA", "DEU", "IND", "IDN", "BRA"]:
    #             color[wh] = r_i/18
    df_hier['color'] = color
    dfs.append(df_hier)
dfs.append(df_hier)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1248\2824324051.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1248\2824324051.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dfg = df.groupby(levels[i:]).sum()
C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1248\2824324051.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1248\2824324051.py:24: FutureWarning: The frame.append method is depre

In [14]:
fig = make_subplots(1, 3, specs=[[{"type": "domain"}, {"type": "domain"}, {"type": "domain"}]],
    horizontal_spacing = 0.02,
    vertical_spacing=0.02,)

for i in range(3):
    row = [1, 1, 1, 2][i]
    col = [1, 2, 3, 2][i]
    app = ['GF', "PC", "PCC", "PCC"][i]
    dfh = dfs[i]
    fig.add_trace(go.Sunburst(
        labels=dfh.id,
        parents=dfh.parent, #df.CONT[:4],
        values=dfh.value, #df.GF[:5],
        branchvalues='total',
        hovertemplate='<b>%{label} </b> <br>Budget: %{value}',
        name='',
        marker=dict(
            colors=dfh.color,
            colorscale='Spectral',
            cmid=0.5),
    ), row, col)
fig.update_traces(sort=False, selector=dict(type='sunburst')) 
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0), height=700, width=2100)
#fig.update_traces(leaf=dict(opacity = 0.5))
fig.show()

In [15]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/Figure_2.svg")
fig.write_image("X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/Figure_2.png", scale=3)

import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/Figure_2.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/Figure_2.html', 'a') as f:
    #f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))